In [35]:
import logging

logging.basicConfig(level=logging.INFO,  # 设置日志级别
                    format='%(asctime)s - %(levelname)s - %(message)s')  # 设置输出格式

In [36]:
import psycopg2
import csv
import logging

logging.getLogger().handlers = []
logging.basicConfig(level=logging.INFO,  # 设置日志级别
                    format='%(asctime)s - %(levelname)s - %(message)s')  # 设置输出格式

DB_CONFIG = {
    "dbname": "chatgis",
    "user": "chatgis",
    "password": "chatgis123",
    "host": "localhost",
    "port": "5432"
}

tables = {}

try:
    # 连接数据库
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    query = f"""
        SELECT matviewname 
        FROM pg_matviews 
        WHERE schemaname = 'california';
        """
    
    query_fields = """
        SELECT attname AS column_name, atttypid::regtype AS data_type
        FROM pg_catalog.pg_attribute
        WHERE attrelid = 'california.%s'::regclass
        AND attnum > 0;
        """
    cur.execute(query)
    rows = cur.fetchall()

    for row in rows:
        logging.debug(f"Table: {row[0]}")
        table_name = row[0]
        tables[table_name] = []
        cur.execute(query_fields % table_name.lower())
        table_fields = cur.fetchall()
        for field in table_fields:
            logging.debug(f"    {field}")
            tables[table_name].append(field)

except Exception as e:
    print("发生错误:", e)

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

logging.debug(f"Tables: {tables}")
logging.info(f"Table information collected!")

2025-03-19 16:39:45,645 - INFO - Table information collected!


In [37]:
import logging
import psycopg2

logging.getLogger().handlers = []
logging.basicConfig(level=logging.DEBUG,  # 设置日志级别
                    format='%(asctime)s - %(levelname)s - %(message)s')  # 设置输出格式

fclasses_table = {}
fclasses_list = []

try:
    # 连接数据库
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    query_fclass = """SELECT DISTINCT fclass FROM california.%s"""

    for t in sorted(tables.keys()):
        logging.debug(f"{t} {tables[t]}")
        cur.execute(query_fclass % t)
        fclasses = cur.fetchall()
        fclasses_table[t] = []
        for i in fclasses:
            fclasses_table[t].append(i[0])
            logging.debug(f"fclass: {t}, {i[0]}")
        fclasses_list.append(set(fclasses_table[t]))
        logging.debug(f"fclasses: {fclasses_table[t]}")

    fclass_intersection = set.intersection(*fclasses_list[1:])
    
except Exception as e:
    print("发生错误:", e)

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()



logging.info(f"fclass intersection: {set.intersection(*fclasses_list[1:])}")
# logging.info(f"fclass union: {set.union(*fclasses_list[1:])}")
# logging.info(f"fclass difference: {set.difference(*fclasses_list[1:])}")
logging.info(f"fclass collected!")

2025-03-19 16:39:48,212 - DEBUG - buildings_a [('gid', 'integer'), ('osm_id', 'character varying'), ('code', 'smallint'), ('fclass', 'character varying'), ('name', 'character varying'), ('type', 'character varying'), ('geom', 'geometry')]
2025-03-19 16:39:59,138 - DEBUG - fclass: buildings_a, building
2025-03-19 16:39:59,141 - DEBUG - fclasses: ['building']
2025-03-19 16:39:59,141 - DEBUG - landuse_a [('gid', 'integer'), ('osm_id', 'character varying'), ('code', 'smallint'), ('fclass', 'character varying'), ('name', 'character varying'), ('geom', 'geometry')]
2025-03-19 16:39:59,786 - DEBUG - fclass: landuse_a, allotments
2025-03-19 16:39:59,787 - DEBUG - fclass: landuse_a, cemetery
2025-03-19 16:39:59,787 - DEBUG - fclass: landuse_a, commercial
2025-03-19 16:39:59,787 - DEBUG - fclass: landuse_a, farmland
2025-03-19 16:39:59,788 - DEBUG - fclass: landuse_a, farmyard
2025-03-19 16:39:59,788 - DEBUG - fclass: landuse_a, forest
2025-03-19 16:39:59,788 - DEBUG - fclass: landuse_a, grass
2

In [ ]:
import logging
import psycopg2

logging.getLogger().handlers = []
logging.basicConfig(level=logging.DEBUG,  # 设置日志级别
                    format='%(asctime)s - %(levelname)s - %(message)s')  # 设置输出格式


transfer_fclasses_table = """
    INSERT INTO california_tables.%s 
    SELECT *
    FROM california.%s
    WHERE fclass = '%s';
"""

create_fclass_table = """
    CREATE TABLE IF NOT EXISTS california_tables.%s (LIKE california.%s INCLUDING ALL);
"""

drop_existing_fclass_table = """
    DROP TABLE IF EXISTS california_tables.%s;
"""

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    fclass_num = 0
    for t in sorted(tables.keys()):
        logging.info("正在处理表：{}".format(t))
        
        for fclass in fclasses_table[t]:
            logging.debug("\t正在删除已存在的表格: {}".format(fclass + "_a" if t.endswith("_a") else fclass))
            logging.debug(drop_existing_fclass_table % (fclass + "_a" if t.endswith("_a") else fclass))
            cursor.execute(drop_existing_fclass_table % (fclass + "_a" if t.endswith("_a") else fclass))
            # conn.commit()

            logging.debug("\t正在创建字段类别: {}".format(fclass + "_a" if t.endswith("_a") else fclass))
            logging.debug(create_fclass_table % (fclass + "_a" if t.endswith("_a") else fclass, t))
            cursor.execute(create_fclass_table % (fclass + "_a" if t.endswith("_a") else fclass, t))
            # conn.commit()

            logging.debug("\t\t正在复制数据...")
            logging.debug(transfer_fclasses_table % (fclass + "_a" if t.endswith("_a") else fclass, t, fclass))
            cursor.execute(transfer_fclasses_table % (fclass + "_a" if t.endswith("_a") else fclass, t, fclass))
            # conn.commit()
            logging.debug("\t\t已完成表格{}的fclass类型: {}".format(t, fclass))

            fclass_num += 1

            # for f in tables[t][fclass]:

except psycopg2.Error as e:
    logging.error("连接数据库失败：{}".format(e))

finally:
    if conn:
        conn.commit()
        cursor.close()
        conn.close()

logging.info("数据处理完成！共处理了{}个表格。".format(len(tables.keys())))
logging.info("共处理了{}个fclass类别。".format(fclass_num))


2025-03-19 16:46:25,780 - INFO - 正在处理表：buildings_a
2025-03-19 16:46:25,781 - DEBUG - 	正在删除已存在的表格: building_a
2025-03-19 16:46:25,781 - DEBUG - 
    DROP TABLE IF EXISTS california_tables.building;
_a
2025-03-19 16:46:25,786 - ERROR - 连接数据库失败：syntax error at or near "_a"
LINE 3: _a
        ^

2025-03-19 16:46:25,787 - INFO - 数据处理完成！共处理了18个表格。
2025-03-19 16:46:25,788 - INFO - 共处理了0个fclass类别。


In [29]:
import logging
import psycopg2
import pandas as pd

logging.getLogger().handlers = []
logging.basicConfig(level=logging.INFO,  # 设置日志级别
                    format='%(asctime)s - %(levelname)s - %(message)s')  # 设置输出格式

get_columns_sql = """

SELECT table_schema, table_name, column_name
FROM information_schema.columns
WHERE table_schema = 'california_tables';

"""

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    cursor.execute(get_columns_sql)
    columns = cursor.fetchall()

    for column in columns:
        logging.debug(column)

except psycopg2.Error as e:
    logging.error("连接数据库失败：{}".format(e))

finally:
    if conn:
        cursor.close()
        conn.close()
        logging.info("数据库连接已关闭")

columns_df = pd.DataFrame(columns, columns=['table_schema', 'table_name', 'column_name'])

columns_df.to_csv('columns.csv', index=False)

logging.info(f"共处理{len(columns)}个字段")

2025-03-19 16:17:03,782 - INFO - 数据库连接已关闭
2025-03-19 16:17:03,847 - INFO - 共处理3074个字段


In [18]:
import os
print(os.getcwd())

/Users/liuyucheng/Desktop/DSCI551/Project/code/PostGIS


In [34]:
import pandas as pd
import logging
import pandas as pd

logging.getLogger().handlers = []
logging.basicConfig(level=logging.DEBUG,  # 设置日志级别
                    format='%(asctime)s - %(levelname)s - %(message)s')  # 设置输出格式

df = pd.read_csv('columns.csv')

for t, cs in fclasses_table.items():
    for c in cs:
        logging.debug(f'Processing {c} in {t}')
        sub_df = df[df['table_name'] == c + "_a" if t.endswith("_a") else df['table_name'] == c]
        sub_df.to_csv(f'../../data/metadata/{ c + "_a" if t.endswith("_a") else c }.csv', index=False)
        logging.debug(f'{c} has been saved to metadata/{c}.csv')

logging.info('All columns have been saved to metadata/')

2025-03-19 16:24:23,449 - DEBUG - Processing building in buildings_a


2025-03-19 16:24:23,454 - DEBUG - building has been saved to metadata/building.csv
2025-03-19 16:24:23,454 - DEBUG - Processing allotments in landuse_a
2025-03-19 16:24:23,455 - DEBUG - allotments has been saved to metadata/allotments.csv
2025-03-19 16:24:23,456 - DEBUG - Processing cemetery in landuse_a
2025-03-19 16:24:23,457 - DEBUG - cemetery has been saved to metadata/cemetery.csv
2025-03-19 16:24:23,457 - DEBUG - Processing commercial in landuse_a
2025-03-19 16:24:23,458 - DEBUG - commercial has been saved to metadata/commercial.csv
2025-03-19 16:24:23,458 - DEBUG - Processing farmland in landuse_a
2025-03-19 16:24:23,459 - DEBUG - farmland has been saved to metadata/farmland.csv
2025-03-19 16:24:23,459 - DEBUG - Processing farmyard in landuse_a
2025-03-19 16:24:23,460 - DEBUG - farmyard has been saved to metadata/farmyard.csv
2025-03-19 16:24:23,460 - DEBUG - Processing forest in landuse_a
2025-03-19 16:24:23,461 - DEBUG - forest has been saved to metadata/forest.csv
2025-03-19 

In [32]:
# Example string
text = "San Diego"

# Check if the string ends with "Diego"
if text.endswith("Diego"):
    print("The string ends with 'Diego'.")
else:
    print("The string does not end with 'Diego'.")

The string ends with 'Diego'.


In [21]:
fclasses_table

{'buildings_a': ['building'],
 'landuse_a': ['allotments',
  'cemetery',
  'commercial',
  'farmland',
  'farmyard',
  'forest',
  'grass',
  'heath',
  'industrial',
  'meadow',
  'military',
  'nature_reserve',
  'orchard',
  'park',
  'quarry',
  'recreation_ground',
  'residential',
  'retail',
  'scrub',
  'vineyard'],
 'natural': ['beach',
  'cave_entrance',
  'cliff',
  'peak',
  'spring',
  'tree',
  'volcano'],
 'natural_a': ['cave_entrance',
  'spring',
  'glacier',
  'volcano',
  'cliff',
  'beach'],
 'places': ['county',
  'region',
  'city',
  'town',
  'village',
  'suburb',
  'island',
  'farm',
  'hamlet',
  'locality'],
 'places_a': ['county',
  'region',
  'city',
  'town',
  'village',
  'suburb',
  'farm',
  'island',
  'hamlet',
  'locality'],
 'pofw': ['christian_anglican',
  'muslim_sunni',
  'christian_protestant',
  'muslim',
  'taoist',
  'christian_lutheran',
  'christian_orthodox',
  'christian_methodist',
  'buddhist',
  'christian_catholic',
  'hindu',
  '